<a href="https://colab.research.google.com/github/sachin-p4j/datavita/blob/main/app.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from flask import Flask, render_template, request, redirect, url_for
from werkzeug.utils import secure_filename
import os
from your_script import process_excel  # Import your existing Python script

app = Flask(__name__)
app.config['UPLOAD_FOLDER'] = 'input_files'
app.config['OUTPUT_FOLDER'] = 'output_files'
app.config['ALLOWED_EXTENSIONS'] = {'xlsx'}

def allowed_file(filename):
    return '.' in filename and filename.rsplit('.', 1)[1].lower() in app.config['ALLOWED_EXTENSIONS']

@app.route('/')
def index():
    return render_template('index.html')

@app.route('/upload', methods=['POST'])
def upload_file():
    if 'file' not in request.files:
        return redirect(request.url)

    file = request.files['file']

    if file.filename == '':
        return redirect(request.url)

    if file and allowed_file(file.filename):
        filename = secure_filename(file.filename)
        file_path = os.path.join(app.config['UPLOAD_FOLDER'], filename)
        file.save(file_path)

        # Process the uploaded file using your script
        output_file_path = process_excel(file_path, app.config['OUTPUT_FOLDER'])

        return redirect(url_for('download_file', filename=output_file_path))

    return redirect(request.url)

@app.route('/download/<filename>')
def download_file(filename):
    return send_from_directory(app.config['OUTPUT_FOLDER'], filename, as_attachment=True)

if __name__ == '__main__':
    app.run(debug=True)
